In [1]:
import cudaq
from cudaq import spin
import numpy as np
from math import pi
from typing import List
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, NullLocator
import matplotlib.patches as mpatches
import os
import pandas as pd
import pickle

In [2]:
for i in np.ndindex(2, 2, 2, 2):
    print(np.array(i))

ss = np.array([0, 1])
a = np.array(np.ndindex(2, 2, 2, 2))
# print(ss[a])

[0 0 0 0]
[0 0 0 1]
[0 0 1 0]
[0 0 1 1]
[0 1 0 0]
[0 1 0 1]
[0 1 1 0]
[0 1 1 1]
[1 0 0 0]
[1 0 0 1]
[1 0 1 0]
[1 0 1 1]
[1 1 0 0]
[1 1 0 1]
[1 1 1 0]
[1 1 1 1]


In [3]:
state = np.random.get_state()

np.random.uniform(0, 1, (2, 2))
print(np.random.randn(5))

np.random.set_state(state)
np.random.uniform(-2, 2, (2, 2))
print(np.random.randn(5))


[ 0.23857048 -0.44061487 -0.83550717 -0.54705911 -1.0524233 ]
[ 0.23857048 -0.44061487 -0.83550717 -0.54705911 -1.0524233 ]


In [4]:
a = 1.0
print(a.is_integer())

True


In [5]:
a = np.arange(25).reshape(5, 5)
b = np.array([1, 3, 1, 2, 1])
bb = np.diag(b)
print(a)
print(bb @ a @ bb)

print((b[None, :] * a) * b[:, None])

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 0  3  2  6  4]
 [15 54 21 48 27]
 [10 33 12 26 14]
 [30 96 34 72 38]
 [20 63 22 46 24]]
[[ 0  3  2  6  4]
 [15 54 21 48 27]
 [10 33 12 26 14]
 [30 96 34 72 38]
 [20 63 22 46 24]]


In [6]:
a = np.random.choice(50, 7, replace=False)
print(a)

[10 30 44 34 25 33 31]


In [7]:
df = pd.DataFrame({
    "A": [4, 1, 3, 2],
    "B": [40, 10, 30, 20]
})

df.sort_values(by="A", inplace=True)
print(df)

   A   B
1  1  10
3  2  20
2  3  30
0  4  40


In [8]:
df = pd.read_csv("./experiments_plateau_X/exp_0_L0.001_q1/report.csv")
print(df)
S = df[df["Assets"] == 7].shape[0]
N = df.loc[df["Assets"] == 7, "N"]
print(S, N)

   Assets  Qubits       N     Sum_1     Sum_2     Coeff
0     3.0     9.0  2000.0 -0.589625  3.507773  0.000037
1     4.0    12.0  2000.0 -0.589621  3.507773  0.000037
2     5.0    15.0  2000.0 -0.589623  3.507772  0.000037
3     6.0    18.0  2000.0 -0.589626  3.507771  0.000037
0 Series([], Name: N, dtype: float64)


In [ ]:
a = spin.z(2)   

print(a.to_json())

[1.0, 1.0, 0.0, 1.0, 2.0, 1.0]
